Dans ce TP, nous considérons des trajets en vélo partagé (similaire au vélib) en Californie. Deux jeux de données sont fournis : l'un qui contient les stations de vélo, l'autre, les trajets à vélo. Les déplacements à vélo se font d'une station à l'autre.

Charger le fichier stationData.csv dans un DataFrame station_df et le fichier tripData.csv dans un DataFrame trip_df. Pour chaque Dataframe, il vous faudra demander une inférence des schémas et indiquer que la première ligne est un en-tête.

In [0]:
station_df = spark.read.option(
    "header", "true"
).option(
    "inferSchema", "true"
).csv("/FileStore/tables/stationData.csv")

station_df_2 = spark.read.options(
    header=True,
    inferSchema=True,
).csv("/FileStore/tables/stationData.csv")

trip_df = spark.read.option(
    "header", "true"
).option(
    "inferSchema", "true"
).csv("/FileStore/tables/tripData.csv")

In [0]:
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.2 
 Master 
 local[8] 
 AppName 
 Databricks Shell

Afficher les schémas des 2 DataFrames.

In [0]:
station_df_2.printSchema()
trip_df.printSchema()

root
 |-- station_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- long: double (nullable = true)
 |-- dockcount: integer (nullable = true)
 |-- landmark: string (nullable = true)
 |-- installation: date (nullable = true)

root
 |-- TripID: integer (nullable = true)
 |-- Duration: integer (nullable = true)
 |-- StartDate: string (nullable = true)
 |-- StartStation: string (nullable = true)
 |-- StartTerminal: integer (nullable = true)
 |-- EndDate: string (nullable = true)
 |-- EndStation: string (nullable = true)
 |-- EndTerminal: integer (nullable = true)
 |-- Bike#: integer (nullable = true)
 |-- SubscriberType: string (nullable = true)
 |-- ZipCode: string (nullable = true)



Créer une vue pour chaque DataFrame.

In [0]:
station_df.createOrReplaceTempView("station_view")
trip_df.createOrReplaceTempView("trip_view")

Trouver deux façons de calculer le nombre de trajets, l'une en appelant une méthode sur trip_df directement, l'autre en rédigeant une requête SQL de la vue correspondant au DataFrame tripData.

In [0]:
print(trip_df.count())
spark.sql("""SELECT count(*) FROM trip_view""").show()

354152
+--------+
|count(1)|
+--------+
|  354152|
+--------+



Ecrire une requête permettant de compter le nombre de trajets qui démarrent et se terminent à la même station.

In [0]:
spark.sql("""SELECT count(*) FROM trip_view WHERE StartStation=EndStation""").show()

+--------+
|count(1)|
+--------+
|   10276|
+--------+



On souhaite désormais obtenir l’id des stations associées à ces trajets. Ecrire une requête renvoyant la liste des terminaux concernés ainsi que le nombre de trajets pour chacun de ces terminaux. Trier le résultat par ordre décroissant de nombre de trajets.
<br>Exemple de sortie :
<br>+--------+--------+
<br>|terminal|count(1)|
<br>+--------+--------+
<br>| 60| 850|
<br>| 50| 708|
<br>| 35| 348|
<br>| 76| 320|
<br>| 74| 307|
<br>(La station 60 est la plus concernée par ces trajets cycliques, avec 850 de ces trajets.)

In [0]:
spark.sql(
    """
    SELECT StartTerminal as terminal, count(*)
    FROM trip_view
    WHERE StartStation=EndStation
    GROUP BY terminal
    ORDER BY count(*) DESC
    """
).show()

+--------+--------+
|terminal|count(1)|
+--------+--------+
|      60|     850|
|      50|     708|
|      35|     348|
|      76|     320|
|      74|     307|
|      39|     296|
|      61|     280|
|      67|     277|
|      71|     268|
|      70|     260|
|      28|     254|
|      48|     248|
|      54|     230|
|      69|     227|
|      42|     213|
|      73|     200|
|      57|     197|
|      64|     194|
|       3|     189|
|      72|     181|
+--------+--------+
only showing top 20 rows



In [0]:
from pyspark.sql.functions import col, desc

cyclic_count = trip_df.select("StartTerminal").filter(col("StartStation") == col("EndStation")).groupBy("StartTerminal").count().orderBy(desc("count"))
cyclic_count.show()

+-------------+-----+
|StartTerminal|count|
+-------------+-----+
|           60|  850|
|           50|  708|
|           35|  348|
|           76|  320|
|           74|  307|
|           39|  296|
|           61|  280|
|           67|  277|
|           71|  268|
|           70|  260|
|           28|  254|
|           48|  248|
|           54|  230|
|           69|  227|
|           42|  213|
|           73|  200|
|           57|  197|
|           64|  194|
|            3|  189|
|           72|  181|
+-------------+-----+
only showing top 20 rows



Dans la requête précédente, nous avons oublié un élément qui nous importe. Nous souhaitons compléter le résultat en indiquant le nombre de docks (dockcount) des stations concernées.
<br>Exemple de sortie :
<br>+--------+---------+--------+
<br>|terminal|dockcount|count(1)|
<br>+--------+---------+--------+
<br>| 60| 15| 850|
<br>| 50| 23| 708|
<br>| 35| 11| 348|
<br>| 76| 19| 320|
<br>| 74| 23| 307|
<br>Mettre à jour la requête.

In [0]:
# Votre code ici

Rédiger les 2 requêtes précédentes avec le DSL de DataFrame.

In [0]:
from pyspark.sql.functions import col, desc

In [0]:
res = trip_df.join(
  station_df,
  col("StartTerminal") == col("station_id"),
).select(
  "StartTerminal",
  "dockCount",
).filter(
  col("StartStation") == col("EndStation")
).groupBy(
  "StartTerminal",
  "dockCount"
).count(
).orderBy(
  desc("count")
)

res.show()

+-------------+---------+-----+
|StartTerminal|dockCount|count|
+-------------+---------+-----+
|           60|       15|  850|
|           50|       23|  708|
|           35|       11|  348|
|           76|       19|  320|
|           74|       23|  307|
|           39|       19|  296|
|           61|       27|  280|
|           67|       27|  277|
|           71|       19|  268|
|           70|       19|  260|
|           28|       23|  254|
|           48|       15|  248|
|           54|       15|  230|
|           69|       23|  227|
|           42|       15|  213|
|           73|       15|  200|
|           57|       15|  197|
|           64|       15|  194|
|            3|       15|  189|
|           72|       23|  181|
+-------------+---------+-----+
only showing top 20 rows



Observer le plan d’exécution des requêtes.

In [0]:
res.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- == Current Plan ==
   Sort [count#629L DESC NULLS LAST], true, 0
   +- Exchange rangepartitioning(count#629L DESC NULLS LAST, 200), ENSURE_REQUIREMENTS, [plan_id=1600]
      +- Project [StartTerminal#346, dockCount#284, count(1)#628L AS count#629L]
         +- HashAggregate(keys=[StartTerminal#346, dockCount#284], functions=[finalmerge_count(merge count#643L) AS count(1)#628L])
            +- ShuffleQueryStage 1, Statistics(sizeInBytes=1971.5 MiB)
               +- Exchange hashpartitioning(StartTerminal#346, dockCount#284, 200), ENSURE_REQUIREMENTS, [plan_id=1594]
                  +- *(2) HashAggregate(keys=[StartTerminal#346, dockCount#284], functions=[merge_count(merge count#643L) AS count#643L])
                     +- *(2) Project [StartTerminal#346, dockCount#284, count#643L]
                        +- *(2) BroadcastHashJoin [StartTerminal#346], [station_id#280], Inner, BuildRight, false, true
                           